# Прогнозирование с ARMA (пакет [`pmdarima`](https://alkaline-ml.com/pmdarima/index.html#))

__Важно__: $ARMA$ рассматривается как частный случай $ARIMA$: 

$$
	ARMA(p,q)=ARIMA(p,0,q).
$$

In [ ]:
import numpy as np
import pandas as pd

import pmdarima as pm

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`WGS10YR`](https://fred.stlouisfed.org/series/WGS10YR)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`



In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01', end='2023-12-31')

Пусть `y` – первая разность ставки (не забудем удалить пропущенные наблюдения)

In [ ]:
y = rate.diff().dropna()

## Прогнозирование с ARMA (заданного порядка)

Построим прогноз для `y` на 10 периодов используя модель ARMA(2,2) 

Спецификация

$$
	y_t=\alpha_0+\phi_1y_{t-1}+\phi_2y_{t-2}+u_1+\theta_1 u_{t-1}+\theta_2 u_{t-2}
$$

численный прогноз

In [ ]:
arma = pm.ARIMA(order=(2,0,2), trend='c')
# подгонка модели и прогноз
forecasts = arma.fit_predict(y, n_periods=10)
forecasts

In [ ]:
plt.plot(y.tail(30))
plt.plot(forecasts)

plt.show()

Доверительный интервал

In [ ]:
forecasts, conf_int = arma.predict(n_periods=10, return_conf_int=True, alpha=0.05)
forecasts, conf_int

In [ ]:
plt.plot(y.tail(30))
plt.plot(forecasts)
plt.plot(pd.DataFrame(conf_int, index=forecasts.index))

plt.show()

## Прогнозирование с ARMA ("оптимальный" порядок)

Построим прогноз для `y` на 10 периодов используя модель ARMA(p,q), "оптимальный" порядок которой выбран по информационному критерию 

In [ ]:
arma_opt= pm.auto_arima(y, start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='bic', stationary=True, seasonal=False)
forecasts =arma_opt.predict(n_periods=10)
forecasts

In [ ]:
arma_opt.get_params()['order']

In [ ]:
plt.plot(y.tail(30))
plt.plot(forecasts)

plt.show()